In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
from gensim import corpora

In [2]:

from scipy import spatial

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

In [3]:
dirname = 'data'
files = os.listdir(dirname)
temp = list(map(lambda name: os.path.join(dirname, name), files))

In [4]:
temp[0]

'data\\bank_data_test.csv'

собираем тренирскую подборку

In [5]:
train_df = pd.read_csv(temp[1], sep=',')
test_df = pd.read_csv(temp[0], sep=',')

нормализуем некоторые столбцы

In [6]:
scaler = StandardScaler()
test_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']] = scaler.fit_transform(test_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']])
train_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']] = scaler.fit_transform(train_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']])

текстовые столбцы приводим к числовому формату

In [7]:
def family(strin):
    if strin == "FRIEND" or strin == 'Друг':
        return 1
    elif strin =='MOTHER' or strin == 'Мать' or strin == 'мать':
        return 2
    elif strin == 'RELATIVE' or strin == 'Близкий ро':
        return 3
    elif strin == 'OTHER' or strin == 'Дальний ро':
        return 4
    elif strin == 'BROTHER' or strin == 'Брат':
        return 5
    elif strin == 'SISTER' or strin == 'Сестра':
        return 6
    elif strin == 'FATHER' or strin == 'Отец':
        return 7
    elif strin == 'DAUGHTER' or strin == 'Дочь':
        return 8
    elif strin == 'SON' or strin == 'Сын':
        return 9
    elif strin == 'мать':
        return 10
    elif strin == 'Жена':
        return 11
    else:
        return np.nan

In [8]:
def alfa1(strin):
    if strin == "M":
        return 1
    elif strin =='D':
        return 2
    elif strin == 'V':
        return 3
    elif strin == 'T':
        return 4
    elif strin == 'm':
        return 5
    elif strin == 'N':
        return 6
    elif strin == 'v':
        return 7
    elif strin == 'W':
        return 8
    elif strin == 'd':
        return 9
    elif strin == 't':
        return 10
    elif strin == 'C':
        return 11
    elif strin == 'w':
        return 12
    else:
        return np.nan

In [9]:
def alfa2(strin):
    if strin == "SO":
        return 1
    elif strin =='JO':
        return 2
    elif strin == 'OTHER':
        return 3
    elif strin == 'RENT':
        return 4
    elif strin == 'NPRIVAT':
        return 5
    else:
        return np.nan

In [10]:
def job(strin):
    if strin == "SPECIALIST":
        return 1
    elif strin =='MANAGER':
        return 2
    elif strin == 'TOP_MANAGER':
        return 3
    elif strin == 'SELF_EMPL':
        return 4
    else:
        return np.nan

In [11]:
def binar(strin):
    if strin == "N":
        return 0
    elif strin =='Y':
        return 1
    else:
        return np.nan

In [12]:
def alfa3(strin):
    if strin == "H":
        return 1
    elif strin =='SS':
        return 2
    elif strin == 'UH':
        return 3
    elif strin == 'S':
        return 4
    elif strin == 'HH':
        return 5
    elif strin == 'h':
        return 6
    elif strin == 's':
        return 7
    elif strin == 'A':
        return 8
    elif strin == 'i':
        return 9
    elif strin == 'US':
        return 10
    elif strin == 'I':
        return 11
    elif strin == 'HI':
        return 12
    elif strin == 'a':
        return 13
    elif strin == 'e':
        return 14
    elif strin == 'E':
        return 15
    elif strin == 'AC':
        return 16
    elif strin == 'AV':
        return 17
    else:
        return np.nan

In [13]:
def typeq(strin):
    if strin == "PRIVATE":
        return 1
    elif strin =='STATE':
        return 2
    elif strin == 'INTER':
        return 3
    elif strin == 'IP':
        return 4
    else:
        return np.nan

In [14]:
def pack(strin):
    if strin == "102":
        return 1
    elif strin =='K01':
        return 2
    elif strin == 'O01':
        return 3
    elif strin == '105':
        return 4
    elif strin == '107':
        return 5
    elif strin == '103':
        return 6
    elif strin == '104':
        return 7
    elif strin == '301':
        return 8
    elif strin == '101':
        return 9
    elif strin == '109':
        return 10
    elif strin == 'M01':
        return 11
    elif strin == '108':
        return 12
    else:
        return np.nan

In [15]:
ss = set(train_df['CLNT_JOB_POSITION'].to_list())
box = {}
i = 0
for s in ss:
    if s not in box:
        box[s] = i
        i+=1

In [16]:
def job1(strin):
    if strin in box:
        return box[strin]
    else:
         return np.nan

In [17]:
def floatinf(df):
    df["CLNT_TRUST_RELATION"] = df["CLNT_TRUST_RELATION"].apply(family)
    df["APP_MARITAL_STATUS"] = df["APP_MARITAL_STATUS"].apply(alfa1)
    df["APP_KIND_OF_PROP_HABITATION"] = df["APP_KIND_OF_PROP_HABITATION"].apply(alfa2)
    df["CLNT_JOB_POSITION_TYPE"] = df["CLNT_JOB_POSITION_TYPE"].apply(job)
    df["APP_DRIVING_LICENSE"] = df["APP_DRIVING_LICENSE"].apply(binar)
    df["APP_EDUCATION"] = df["APP_EDUCATION"].apply(alfa3)
    df["APP_TRAVEL_PASS"] = df["APP_TRAVEL_PASS"].apply(binar)
    df["APP_CAR"] = df["APP_CAR"].apply(binar)
    df["APP_POSITION_TYPE"] = df["APP_POSITION_TYPE"].apply(job)
    df["APP_EMP_TYPE"] = df["APP_EMP_TYPE"].apply(typeq)
    df["APP_COMP_TYPE"] = df["APP_COMP_TYPE"].apply(typeq)
    df["PACK"] = df["PACK"].apply(pack)
    df['CLNT_JOB_POSITION'] = df['CLNT_JOB_POSITION'].apply(job1)
    return df

In [18]:
train_df = floatinf(train_df)
test_df = floatinf(test_df)

удаляем пустые столбцы

In [19]:
train_df.drop(['PRC_ACCEPTS_A_EMAIL_LINK',
'PRC_ACCEPTS_A_POS',
'PRC_ACCEPTS_A_TK',
'PRC_ACCEPTS_A_AMOBILE',
'PRC_ACCEPTS_TK',
'PRC_ACCEPTS_A_MTP',
'CNT_ACCEPTS_TK',
'PRC_ACCEPTS_A_ATM',
'PRC_ACCEPTS_MTP',
'CNT_ACCEPTS_MTP'
], axis=1, inplace=True)

In [20]:
test_df.drop(['PRC_ACCEPTS_A_EMAIL_LINK',
'PRC_ACCEPTS_A_POS',
'PRC_ACCEPTS_A_TK',
'PRC_ACCEPTS_A_AMOBILE',
'PRC_ACCEPTS_TK',
'PRC_ACCEPTS_A_MTP',
'CNT_ACCEPTS_TK',
'PRC_ACCEPTS_A_ATM',
'PRC_ACCEPTS_MTP',
'CNT_ACCEPTS_MTP'
], axis=1, inplace=True)

In [21]:
for i in train_df.columns:
    #df[i] = df[i].astype(np.float32)
    if train_df[i].isnull().any():
        train_df[i].fillna(train_df[i].median(), inplace=True)

In [22]:
for i in test_df.columns:
    #df[i] = df[i].astype(np.float32)
    if test_df[i].isnull().any():
        test_df[i].fillna(test_df[i].median(), inplace=True)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Columns: 106 entries, ID to TARGET
dtypes: float64(96), int64(10)
memory usage: 287.2 MB


In [24]:
train_df.set_index('ID', inplace=True)
test_df.set_index('ID', inplace=True)

In [25]:
test_df.head(3)

,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
400980,0,0.000000,0.128378,0.0,0.3,0.288088,0.000000,0.724026,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
525062,0,0.059381,0.128378,0.0,0.3,0.288088,0.000333,0.724026,2.0,0.0,...,0.0,1.0,0.5,0.5,0.0,0.0,0.5,0.5,0.5,NaN
280316,0,0.070430,0.128378,0.0,0.3,0.288088,0.077876,0.724026,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [26]:
scaler = StandardScaler()
test_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']] = scaler.fit_transform(test_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']])
train_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']] = scaler.fit_transform(train_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']])

In [49]:
train_df.describe().to_excel('fff1.xlsx')

In [23]:
train_df.head(50).to_excel('fff.xlsx')

In [27]:
tarin_df1 = train_df

создаем список характеристик

In [28]:
target = 'TARGET'
features = [x for x in list(tarin_df1.columns) if x != target]
features[0]

'CR_PROD_CNT_IL'

In [52]:
# Мы использовали последовательную модель, которая представляет собой тип модели, в которой слои укладываются линейно. Первый слой - это слой ввода, где input_dim - это количество функций, а количество выходных единиц равно 16. В скрытом слое мы используем 8 единиц вывода. Наконец, на выходном уровне у нас есть только одна единица вывода, которая возвращает вероятность оттока клиентов. Во входном и скрытом слоях мы использовали функцию активации relu, а на выходе - сигмоидальную функцию активации.
# model = Sequential()
# model.add(Dense(16, input_dim=len(features), activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [53]:

# compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Разбиваем на выборки

In [29]:
X_train, X_test, y_train, y_test = train_test_split(tarin_df1[features], 
                                                    tarin_df1[target], 
                                                    test_size=0.2,
                                                   random_state=21, stratify=tarin_df1[target])

бейслайн модель

In [30]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [47]:
print('AUC for baseline set: %0.4f' % roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1]))

AUC for baseline set: 0.5000


Ищкм наиболее эффектиный метод обучения, через рандом форест

In [32]:
rfc1 = RandomForestClassifier(max_depth=25, n_estimators=80, random_state=21)
rfc1.fit(X_train, y_train)

RandomForestClassifier(max_depth=25, n_estimators=80, random_state=21)

In [43]:
print('AUC for rfc set: %0.4f' %roc_auc_score(y_test, rfc1.predict_proba(X_test)[:, 1]))

AUC for rfc set: 0.8287


mlp модель

In [33]:
mlp = MLPClassifier(hidden_layer_sizes=(32,), random_state=21)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(32,), random_state=21)

In [41]:
print('AUC for rfc set: %0.4f' % roc_auc_score(y_test, mlp.predict_proba(X_test)[:, 1]))

AUC for rfc set: 0.8209


In [155]:
roc_auc_score(y_train, rfc1.predict_proba(X_train)[:, 1])

0.9961492786318675

грид сетка для фореста

In [32]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': np.arange(21, 25),
              'n_estimators': np.arange(50, 80)}
gs = GridSearchCV(rfc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([21, 22, 23, 24]),
                         'n_estimators': array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])},
             scoring='accuracy')

In [42]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [22],
              'n_estimators': np.arange(50, 70)}
gs = GridSearchCV(rfc, param_grid, scoring='roc_auc', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [22],
                         'n_estimators': array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69])},
             scoring='roc_auc')

In [41]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

а вот и самая эффективная сборка

In [45]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 69}

Оценка нам подходит

In [43]:
roc_auc_score(y_test, gs.best_estimator_.predict_proba(X_test)[:, 1])

0.8312937737551045

нейросеть на keras

In [34]:
# Мы использовали последовательную модель, которая представляет собой тип модели, в которой слои укладываются линейно. Первый слой - это слой ввода, где input_dim - это количество функций, а количество выходных единиц равно 16. В скрытом слое мы используем 8 единиц вывода. Наконец, на выходном уровне у нас есть только одна единица вывода, которая возвращает вероятность оттока клиентов. Во входном и скрытом слоях мы использовали функцию активации relu, а на выходе - сигмоидальную функцию активации.
model = Sequential()
model.add(Dense(16, input_dim=len(features), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [35]:

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=30, batch_size=100)

Epoch 1/30
2842/2842 [==============================] - 5s 1ms/step - loss: 0.2637 - accuracy: 0.9186
Epoch 2/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2558 - accuracy: 0.9186
Epoch 3/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2520 - accuracy: 0.9186
Epoch 4/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2480 - accuracy: 0.9186
Epoch 5/30
2842/2842 [==============================] - 3s 962us/step - loss: 0.2443 - accuracy: 0.9186
Epoch 6/30
2842/2842 [==============================] - 2s 761us/step - loss: 0.2411 - accuracy: 0.9186
Epoch 7/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2388 - accuracy: 0.9186
Epoch 8/30
2842/2842 [==============================] - 3s 996us/step - loss: 0.2368 - accuracy: 0.9186
Epoch 9/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2355 - accuracy: 0.9186
Epoch 10/30
2842/2842 [==============================] - 3s 1ms/step - loss:

In [34]:
test_preds = [x[0] for x in model.predict(X_test)]

2220/2220 [==============================] - 1s 556us/step


In [35]:
print('AUC for keras set: %0.4f' % roc_auc_score(y_test, test_preds))

AUC for keras set: 0.7388


In [37]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['entropy'],
              'max_depth': [22],
              'n_estimators': [69]}
gs = GridSearchCV(rfc, param_grid, scoring='roc_auc', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [22],
                         'n_estimators': [69]},
             scoring='roc_auc')

формируем таблицу результатов

In [38]:
class Table:
    def __init__(self, model: list):
        self.models = model
    def form_table(self):
        res = []
        for i in range(len(self.models)):
            if i == 3:
                auc = self.models[i][4]
            else:
                auc = self.add_auc(self.models[i][0])
            res.append({'librery': self.models[i][1], 'alorithm': self.models[i][2], 'parametrs': self.models[i][3], 'accuracy': self.add_accuracy(self.models[i][0], i), 'auc': auc})
        return(pd.DataFrame(res))
    def add_auc(self, model):
        try:
            test_preds = model.predict_proba(X_test)[:, 1]
        except:
            test_preds = [x[0] for x in model.predict(X_test)]
        return('%0.4f' % roc_auc_score(y_test, test_preds))
    def add_accuracy(self, model, i):
        print()
        # try:
            # test_preds = model.predict_proba(X_test)
        # except:
        print(i)
        if i == 4:
            print('ddddd')
            test_preds = [round(x[0]) for x in model.predict(X_test)]
        else:
            test_preds = model.predict(X_test)
        return('%0.4f' % accuracy_score(y_test, test_preds))

In [39]:
g = Table([(rfc1, 'sklearn.RandomForestClassifier', 'A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.', 'max_depth=25, n_estimators=80, random_state=21'), (mlp, 'sclrean.MLPClassifier', 'This model optimizes the log-loss function using LBFGS or stochastic gradient descent', 'hidden_layer_sizes=(32,), random_state=21'), (dummy_clf, 'skleran.DummyClassifier','DummyClassifier makes predictions that ignore the input features. This classifier serves as a simple baseline to compare against other more complex classifiers. The specific behavior of the baseline is selected with the strategy parameter', 'strategy="most_frequent"'), (gs, 'sklearn.GridSearchCV', 'Exhaustive search over specified parameter values for an estimator. Important members are fit, predict', "'criterion': ['gini', 'entropy'], 'max_depth': [22], 'n_estimators': np.arange(50, 70)", 0.8312), (model, 'keras.Sequential', 'The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs', "loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']")])

In [40]:
fin_df = g.form_table()


0

1

2

3
2220/2220 [==============================] - 2s 587us/step

4
ddddd
2220/2220 [==============================] - 1s 579us/step


In [41]:
fin_df

,librery,alorithm,parametrs,accuracy,auc
0,sklearn.RandomForestClassifier,A random forest is a meta estimator that fits ...,"max_depth=25, n_estimators=80, random_state=21",0.9193,0.8295
1,sclrean.MLPClassifier,This model optimizes the log-loss function usi...,"hidden_layer_sizes=(32,), random_state=21",0.9167,0.8209
2,skleran.DummyClassifier,DummyClassifier makes predictions that ignore ...,"strategy=""most_frequent""",0.9186,0.5000
3,sklearn.GridSearchCV,Exhaustive search over specified parameter val...,"'criterion': ['gini', 'entropy'], 'max_depth':...",0.9194,0.8312
4,keras.Sequential,The sequential API allows you to create models...,"loss='binary_crossentropy', optimizer='adam', ...",0.9191,0.8141


формируем ксв файл для тестовой таблицы

In [ ]:
xx_train = test_df.drop(['TARGET'], axis=1)

In [ ]:
pred = gs.best_estimator_.predict(xx_train)
# csv_df1['predict'] = pred
xx_train['predict'] = pred
xx_train
xy_train = xx_train['predict']
xy_train

ID
400980    1
525062    0
280316    0
496066    0
375031    0
         ..
175305    0
275016    0
277398    0
469164    0
397854    1
Name: predict, Length: 88798, dtype: int64

In [81]:
xy_train.to_csv('data/new_test.csv', sep = ';')

In [55]:
# X_train = np.asarray(X_train).astype('float32')